El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado. A Rusty Bargain le interesa:

la calidad de la predicción;
la velocidad de la predicción;
el tiempo requerido para el entrenamiento

In [22]:
# Librerias

import numpy as np
import pandas as pd
import math
import time

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


Preparación de datos

In [23]:
df = pd.read_csv('car_data.csv')
df.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,0,33775,06/04/2016 19:17
6,01/04/2016 20:48,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,67112,05/04/2016 18:18
7,21/03/2016 18:54,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,21/03/2016 00:00,0,19348,25/03/2016 16:47
8,04/04/2016 23:42,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,04/04/2016 00:00,0,94505,04/04/2016 23:42
9,17/03/2016 10:53,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,17/03/2016 00:00,0,27472,31/03/2016 17:17


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [25]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [26]:
#  Eliminar columnas no relevantes

df_eliminate_columns = df.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)
df_eliminate_columns

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [27]:
# Visualizacion de  valores nulos

df_eliminate_columns.isna().mean()

Price                0.000000
VehicleType          0.105794
RegistrationYear     0.000000
Gearbox              0.055967
Power                0.000000
Model                0.055606
Mileage              0.000000
RegistrationMonth    0.000000
FuelType             0.092827
Brand                0.000000
NotRepaired          0.200791
dtype: float64

In [28]:
# Convertir los NAN en unknown

df_eliminate_columns= df_eliminate_columns.fillna('unknown')
df_eliminate_columns

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,unknown,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,unknown,2005,unknown,0,unknown,20000,1,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [29]:
# Renbombrar columnas

df_eliminate_columns = df_eliminate_columns.rename(columns={'Price': 'price', 'VehicleType': 'vehicle_type', 'RegistrationYear': 'registration_year', 'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model', 'Mileage': 'mileage', 'RegistrationMonth': 'registration_month', 'FuelType': 'fuel_type','Brand': 'brand' ,'NotRepaired': 'not_repaired'})
df_eliminate_columns


,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,unknown,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,unknown,2005,unknown,0,unknown,20000,1,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [30]:
# Contar cantidad de ceros de la columna price  
# equivale 3% de  todos los datos

cero = df_eliminate_columns['price'] == 0
cero.sum()

10772

In [31]:
# Eliminar los  valores en ceros de la columna price

df_eliminate_columns= df_eliminate_columns[df_eliminate_columns['price'] > 0]
df_eliminate_columns

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no
354365,2200,unknown,2005,unknown,0,unknown,20000,1,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [32]:
# Renombrar el dataset

df_mod = df_eliminate_columns.copy()
df_mod

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no
354365,2200,unknown,2005,unknown,0,unknown,20000,1,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [33]:
# Vsualiza las categorias de model

df_mod['model'].value_counts(1).sort_values()

model
serie_1               0.000003
range_rover_evoque    0.000006
serie_3               0.000009
rangerover            0.000012
i3                    0.000023
                        ...   
polo                  0.036752
unknown               0.050993
3er                   0.055818
other                 0.068997
golf                  0.082207
Name: proportion, Length: 251, dtype: float64

Entrenamiento del modelo

In [34]:
df_mod

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no
354365,2200,unknown,2005,unknown,0,unknown,20000,1,unknown,sonstige_autos,unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [35]:
# ONE HOT ENCODING

enc =OneHotEncoder(handle_unknown='ignore', min_frequency= 0.02, sparse_output=False)


enc.fit(df_mod[['model', 'brand','vehicle_type','gearbox','fuel_type','not_repaired']])

df_mod_cat = enc.transform(df_mod[['model', 'brand','vehicle_type','gearbox','fuel_type','not_repaired']])

In [36]:
df_mod_cat_clean = pd.DataFrame(
    df_mod_cat,
    columns=enc.get_feature_names_out(['model', 'brand','vehicle_type','gearbox','fuel_type','not_repaired'])
)

df_mod_cat_clean

,model_3er,model_5er,model_a4,model_astra,model_c_klasse,model_corsa,model_golf,model_other,model_passat,model_polo,...,gearbox_auto,gearbox_manual,gearbox_unknown,fuel_type_gasoline,fuel_type_petrol,fuel_type_unknown,fuel_type_infrequent_sklearn,not_repaired_no,not_repaired_unknown,not_repaired_yes
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
343593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
343594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
343595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
df_mod[['price', 'registration_year', 'power', 'mileage', 'registration_month']].reset_index(drop=True)

,price,registration_year,power,mileage,registration_month
0,480,1993,0,150000,0
1,18300,2011,190,125000,5
2,9800,2004,163,125000,8
3,1500,2001,75,150000,6
4,3600,2008,69,90000,7
...,...,...,...,...,...
343592,1150,2000,0,150000,3
343593,2200,2005,0,20000,1
343594,1199,2000,101,125000,3
343595,9200,1996,102,150000,3


In [38]:
df_encoded = pd.concat(
    [
        df_mod[['price', 'registration_year', 'power', 'mileage', 'registration_month']].reset_index(drop=True),
        df_mod_cat_clean.reset_index(drop=True)
    ],
    axis=1,
)

df_encoded

,price,registration_year,power,mileage,registration_month,model_3er,model_5er,model_a4,model_astra,model_c_klasse,...,gearbox_auto,gearbox_manual,gearbox_unknown,fuel_type_gasoline,fuel_type_petrol,fuel_type_unknown,fuel_type_infrequent_sklearn,not_repaired_no,not_repaired_unknown,not_repaired_yes
0,480,1993,0,150000,0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,18300,2011,190,125000,5,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9800,2004,163,125000,8,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1500,2001,75,150000,6,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,3600,2008,69,90000,7,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343592,1150,2000,0,150000,3,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
343593,2200,2005,0,20000,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
343594,1199,2000,101,125000,3,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
343595,9200,1996,102,150000,3,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


Análisis del modelo

In [39]:
# Escalado de los Datos

scaler = StandardScaler()

# Ajustamos el escalador y transformamos los datos
datos_escalados = scaler.fit_transform(df_encoded)
df_escalado = pd.DataFrame(datos_escalados, columns=df_encoded.columns)

print("\nDatos escalados:")
print(df_escalado)


Datos escalados:
           price  registration_year     power   mileage  registration_month  \
0      -0.902564          -0.141428 -0.589869  0.577340           -1.562360   
1       3.044235           0.088126  0.419815 -0.088953           -0.210344   
2       1.161643          -0.001145  0.276334 -0.088953            0.600866   
3      -0.676653          -0.039404 -0.191310  0.577340            0.060059   
4      -0.211542           0.049867 -0.223194 -1.021764            0.330462   
...          ...                ...       ...       ...                 ...   
343592 -0.754171          -0.052157 -0.589869  0.577340           -0.751151   
343593 -0.521616           0.011608 -0.589869 -2.887387           -1.291957   
343594 -0.743319          -0.052157 -0.053142 -0.088953           -0.751151   
343595  1.028754          -0.103169 -0.047828  0.577340           -0.751151   
343596 -0.255838          -0.026651 -0.058456  0.577340            0.060059   

        model_3er  model_5er  mod

In [40]:
# Segmentacion de Datos

x = df_escalado.drop('price', axis=1)
y = df_escalado['price']

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.25, random_state=12345)

In [41]:
# # Modelo Regresion Lineal

def lineal_regresion(x_train, y_train, x_test, y_test):
    model =  LinearRegression()
        
    inicio_entreno = time.time()
    model.fit(x_train, y_train)
    fin_entreno = time.time()
    tiempo_entrenamiento = fin_entreno - inicio_entreno

    inicio_pred = time.time()
    y_pred = model.predict(x_test)
    fin_pred = time.time()
    tiempo_prediccion = fin_pred - inicio_pred

  
    rmse = root_mean_squared_error(y_test, y_pred)
    return model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion



model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion =lineal_regresion(x_train, y_train,x_test, y_test)

print(f"Modelo de Regresión Lineal entrenado: {model}")
print(f"Número de predicciones realizadas: {len( y_pred)}")
print(f"RECM (Random Forest) calculado: {rmse:.2f}")   
print(f"Tiempo de Entrenamiento: {tiempo_entrenamiento:.4f}")  
print(f"Tiempo de Predicción {tiempo_prediccion:.4f}")  

Modelo de Regresión Lineal entrenado: LinearRegression()
Número de predicciones realizadas: 85900
RECM (Random Forest) calculado: 0.74
Tiempo de Entrenamiento: 1.0377
Tiempo de Predicción 0.0181


In [42]:
# # Modelo Random Forest


def random_forest(x_train, y_train, x_test, y_test):
  
    model = RandomForestRegressor(max_depth = 5, min_samples_split=0.03 )
    
    inicio_entreno = time.time()
    model.fit(x_train, y_train)
    fin_entreno = time.time()
    tiempo_entrenamiento = fin_entreno - inicio_entreno

    inicio_pred = time.time()
    y_pred = model.predict(x_test)
    fin_pred = time.time()
    tiempo_prediccion = fin_pred - inicio_pred
    
    
    rmse = root_mean_squared_error(y_test, y_pred)
    return model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion
  
model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion =random_forest(x_train, y_train, x_test, y_test)

print(f"Modelo de Random Forest entrenado: {model}")
print(f"Número de predicciones realizadas: {len( y_pred)}")
print(f"RECM (Random Forest) calculado: {rmse:.2f}")  
print(f"Tiempo de Entrenamiento: {tiempo_entrenamiento:.4f}")  
print(f"Tiempo de Predicción {tiempo_prediccion:.4f}")  

Modelo de Random Forest entrenado: RandomForestRegressor(max_depth=5, min_samples_split=0.03)
Número de predicciones realizadas: 85900
RECM (Random Forest) calculado: 0.57
Tiempo de Entrenamiento: 88.2946
Tiempo de Predicción 0.4973


In [43]:
# Modelo LightGBM

def lightgbm (x_train, y_train, x_test, y_test):
  
    model = LGBMRegressor(n_estimators=50, learning_rate=0.1, max_depth=10)
 
    inicio_entreno = time.time()
    model.fit(x_train, y_train)
    fin_entreno = time.time()
    tiempo_entrenamiento = fin_entreno - inicio_entreno

    inicio_pred = time.time()
    y_pred = model.predict(x_test)
    fin_pred = time.time()
    tiempo_prediccion = fin_pred - inicio_pred
     
    rmse = root_mean_squared_error(y_test, y_pred)
    return model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion
  
model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion =lightgbm(x_train, y_train, x_test, y_test)

print(f"Modelo de LightGBM entrenado: {model}")
print(f"Número de predicciones realizadas: {len( y_pred)}")
print(f"RECM (LightGBM) calculado: {rmse:.2f}")  
print(f"Tiempo de Entrenamiento: {tiempo_entrenamiento:.4f}")  
print(f"Tiempo de Predicción {tiempo_prediccion:.4f}")  

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037601 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 503
[LightGBM] [Info] Number of data points in the train set: 257697, number of used features: 45
[LightGBM] [Info] Start training from score -0.000136
Modelo de LightGBM entrenado: LGBMRegressor(max_depth=10, n_estimators=50)
Número de predicciones realizadas: 85900
RECM (LightGBM) calculado: 0.42
Tiempo de Entrenamiento: 4.6411
Tiempo de Predicción 0.1086


In [44]:
# Modelo Catboost

def catboost_model(x_train, y_train, x_test, y_test):
    
    model = CatBoostRegressor(
        iterations=50,
        learning_rate=0.1,
        depth=5,
        verbose=0 )
        
    inicio_entreno = time.time()
    model.fit(x_train, y_train)
    fin_entreno = time.time()
    tiempo_entrenamiento = fin_entreno - inicio_entreno

    inicio_pred = time.time()
    y_pred = model.predict(x_test)
    fin_pred = time.time()
    tiempo_prediccion = fin_pred - inicio_pred
    
    rmse = root_mean_squared_error(y_test, y_pred)
    return model, y_pred, rmse, tiempo_entrenamiento, tiempo_prediccion

model, y_pred, rmse , tiempo_entrenamiento, tiempo_prediccion= catboost_model(x_train, y_train, x_test, y_test)


print(f"Modelo de CatBoost entrenado: {model}")
print(f"Número de predicciones realizadas: {len(y_pred)}")
print(f"RECM (CatBoost) calculado: {rmse:.2f}")
print(f"Tiempo de Entrenamiento: {tiempo_entrenamiento:.4f}")  
print(f"Tiempo de Predicción {tiempo_prediccion:.4f}")  




Modelo de CatBoost entrenado: <catboost.core.CatBoostRegressor object at 0x00000169434A4A50>
Número de predicciones realizadas: 85900
RECM (CatBoost) calculado: 0.46
Tiempo de Entrenamiento: 2.2882
Tiempo de Predicción 0.0177


CONCLUSIONES

En el modelo de Regresión Lineal, el RECM: 0.74, Tiempo de entrenamiento: 0.61 segundos y Tiempo de predicción: 0.0110 segundos para 85,900 predicciones.Es el modelo de peor calidad predictiva entre los evaluados. 

El modelo de Random Forest obtuvo un RECM: 0.57, Tiempo de entrenamiento: 78.95 segundos, es decir, 1 minuto y 19 segundos y Tiempo de predicción: 0.3639 segundos. Mostrando una mejora significativa respecto a la Regresión Lineal pero su calidad sigue siendo baja y el tiempo de prediccion el más lento de todos debido a la construcción de múltiples árboles de decisión.

El modelo LightGBM logró el mejor RECM: 0.42,  Tiempo de entrenamiento: muy rápido 1.73 segundos y Tiempo de predicción: rápido 0.0913 segundos. Siendo el modelo con mayor calidad y precisión predictiva
superando a CatBoost y Random Forest. Es uno de los algoritmos de gradient boosting más eficientes.

El modelo CatBoost obtuvo un RECM de 0.46, Tiempo de entrenamiento: rápido (2.57 segundos) y Tiempo de predicción:0.0122 segundos. Este modelo es muy competitivo y cercano a LightGBM pero mucho más eficiente que Random Forest y la Regresión Lineal.

En conclusión, el modelo LightGBM es la opción más sólida ya que ofrece la mejor calidad predictiva, un tiempo de entrenamiento competitivo, y un buen balance entre velocidad y precisión. Destaca como la mejor opción entre los cuatro modelos evaluados.